In [ ]:
# Change the current directory to /content
%cd /content

# Clone the ComfyUI repository from GitHub
!git clone https://github.com/comfyanonymous/ComfyUI.git ./ComfyUI
!git clone https://github.com/city96/ComfyUI-GGUF.git ./ComfyUI/custom_nodes/quantized

In [ ]:
# Change the current directory to /content/ComfyUI
%cd /content/ComfyUI

# Install library
!pip install -q einops==0.8.0 torchsde==0.2.6 diffusers==0.31.0 accelerate==1.1.0 gguf==0.10.0 gradio==5.9.1
!pip install -q huggingface_hub==0.27.0 hf_transfer==0.1.8

In [ ]:
# Download pre-trained models using huggingface-cli

# Enable faster downloads with HF_HUB_ENABLE_HF_TRANSFER
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download martintomov/Hyper-FLUX.1-dev-gguf hyper-flux-16step-Q4_K_M.gguf --local-dir ./models/unet --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download Comfy-Org/sigclip_vision_384 sigclip_vision_patch14_384.safetensors --local-dir ./models/clip_vision --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download Thelocallab/Flux-Dev-Redux flux1-redux-dev.safetensors --local-dir ./models/style_models --local-dir-use-symlinks False # originally, it's from black-forest-labs/FLUX.1-Redux-dev. Check the LICENSE.
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download camenduru/FLUX.1-dev ae.sft --local-dir ./models/vae --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download zer0int/CLIP-GmP-ViT-L-14 ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors --local-dir ./models/clip --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp8_e4m3fn.safetensors --local-dir ./models/clip --local-dir-use-symlinks False

In [ ]:
# Change the current directory to /content/ComfyUI
%cd /content/ComfyUI

import os
import random
import torch
import numpy as np
from PIL import Image
import gradio as gr

# ComfyUI imports
import nodes
from nodes import NODE_CLASS_MAPPINGS
from comfy import model_management
from comfy_extras import nodes_custom_sampler, nodes_flux, nodes_model_advanced

# Custom Node imports
from custom_nodes.quantized.nodes import NODE_CLASS_MAPPINGS as Q_NODE_CLASS_MAPPINGS

# Global model/node references
clip = None
vae = None
unet = None
clip_vision = None
style_model = None

CLIPVisionLoader = None
LoadImage = None
StyleModelLoader = None
CLIPVisionEncode = None
StyleModelApply = None

DualCLIPLoader = None
VAELoader = None
CLIPTextEncode = None
FluxGuidance = None
ModelSamplingFlux = None
BasicGuider = None
KSamplerSelect = None
BasicScheduler = None
SamplerCustomAdvanced = None

RandomNoise = None
EmptyLatentImage = None
VAEDecode = None

In [ ]:
def load_models():
    """
    Loads all the necessary ComfyUI nodes and models (CLIP, VAE, UNet, CLIP Vision, StyleModel).
    """
    global clip, vae, unet
    global clip_vision, style_model

    global CLIPVisionLoader, LoadImage, StyleModelLoader
    global CLIPVisionEncode, StyleModelApply
    global DualCLIPLoader, VAELoader, CLIPTextEncode
    global FluxGuidance, ModelSamplingFlux, BasicGuider
    global KSamplerSelect, BasicScheduler, SamplerCustomAdvanced
    global RandomNoise, EmptyLatentImage, VAEDecode

    # Instantiate node classes
    CLIPVisionLoader = NODE_CLASS_MAPPINGS["CLIPVisionLoader"]()
    LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
    StyleModelLoader = NODE_CLASS_MAPPINGS["StyleModelLoader"]()
    CLIPVisionEncode = NODE_CLASS_MAPPINGS["CLIPVisionEncode"]()
    StyleModelApply = NODE_CLASS_MAPPINGS["StyleModelApply"]()

    DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
    VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
    CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
    FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
    ModelSamplingFlux = nodes_model_advanced.NODE_CLASS_MAPPINGS["ModelSamplingFlux"]()
    BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
    KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
    BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
    SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()

    RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
    EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
    VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()

    # Load essential models
    with torch.inference_mode():
        clip = DualCLIPLoader.load_clip(
            "t5xxl_fp8_e4m3fn.safetensors",
            "ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors",
            "flux"
        )[0]
        vae = VAELoader.load_vae("ae.sft")[0]
        clip_vision = CLIPVisionLoader.load_clip("sigclip_vision_patch14_384.safetensors")[0]
        style_model = StyleModelLoader.load_style_model("flux1-redux-dev.safetensors")[0]

    # Load the quantized UNet
    UNETLoader = Q_NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
    with torch.inference_mode():
        unet = UNETLoader.load_unet("hyper-flux-16step-Q4_K_M.gguf")[0]


def closestNumber(n, m):
    """
    Finds the closest multiple of m to the number n.
    """
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)

    if abs(n - n1) < abs(n - n2):
        return n1
    return n2


def generate_image(image, prompt, width, height, seed):
    """
    Generates an image by applying a style model based on an input image,
    text prompt, and specified dimensions/seed.
    """
    with torch.inference_mode():
        steps = 16
        sampler_name = "euler"
        scheduler = "simple"

        # Save the input image to disk so we can load it with ComfyUI's node
        image.save("/content/ComfyUI/input/image.png")
        loaded_image = LoadImage.load_image("image.png")[0]

        clip_vision_output = CLIPVisionEncode.encode(
            clip_vision=clip_vision,
            image=loaded_image,
            crop="none"
        )[0]

        cond = CLIPTextEncode.encode(text=prompt, clip=clip)[0]
        cond_f = FluxGuidance.append(conditioning=cond, guidance=3.5)[0]

        cond_ff = StyleModelApply.apply_stylemodel(
            conditioning=cond_f,
            style_model=style_model,
            clip_vision_output=clip_vision_output,
            strength=1.0,
            strength_type="multiply"
        )[0]

        noise = RandomNoise.get_noise(seed)[0]
        new_unet = ModelSamplingFlux.patch(
            model=unet,
            width=width,
            height=height,
            max_shift=1.15,
            base_shift=0.5
        )[0]

        sampler = KSamplerSelect.get_sampler(sampler_name=sampler_name)[0]
        guider = BasicGuider.get_guider(model=new_unet, conditioning=cond_ff)[0]
        sigmas = BasicScheduler.get_sigmas(
            model=new_unet,
            scheduler=scheduler,
            steps=steps,
            denoise=1.0
        )[0]

        latent_image = EmptyLatentImage.generate(
            closestNumber(width, 16),
            closestNumber(height, 16)
        )[0]

        sample, _ = SamplerCustomAdvanced.sample(
            noise=noise,
            guider=guider,
            sampler=sampler,
            sigmas=sigmas,
            latent_image=latent_image
        )
        model_management.soft_empty_cache()

        decoded = VAEDecode.decode(vae=vae, samples=sample)[0].detach()
        output_image = Image.fromarray(
            np.array(decoded * 255, dtype=np.uint8)[0]
        )
        return output_image


def on_generate_click(image, prompt, width, height, seed):
    """
    Callback function for the '이미지 생성하기' button in Gradio.
    Attempts to generate the image and returns the result and status.
    """
    if not prompt.strip():
        return None, "❌ 텍스트를 입력해주세요.", None
    if not image:
        return None, "❌ 이미지를 입력해주세요.", None
    try:
        generated = generate_image(image, prompt, width, height, seed)
        return generated, "✅ 성공적으로 이미지를 생성했습니다.", seed
    except Exception as e:
        return None, f"❌ 에러 발생: {str(e)}", None


def roll_seed():
    """
    Generates a random seed.
    """
    return random.randint(0, 2**32 - 1)


def create_interface():
    css = """
    footer {
        visibility: hidden;
    }
    """

    interface = gr.Interface(
        fn=on_generate_click,
        inputs=[
            gr.Image(
                label="🖼️ 이미지",
                type="pil",
                height="100%",
            ),
            gr.Textbox(
                label="📝 텍스트",
                placeholder="원하는 장면을 영어로 입력해주세요...",
                lines=3
            ),
            gr.Slider(
                label="📏 가로 (pixels)",
                minimum=64,
                maximum=1280,
                step=64,
                value=1024
            ),
            gr.Slider(
                label="📏 세로 (pixels)",
                minimum=64,
                maximum=1280,
                step=64,
                value=1024
            ),
            gr.Slider(0, 2**32 -1, 2024, label="🎲 시드", step=1)
        ],
        outputs=[
            gr.Image(label="🖼️ 생성한 이미지"),
            gr.Textbox(label="🔄 현재 상태", interactive=False),
            gr.Number(label="📌 사용된 시드", value=42, precision=0, interactive=False)
        ],
        title="🖼️ 플럭스 활용하기",
        theme='JohnSmith9982/small_and_pretty',
        css=css,
        allow_flagging="never",
    )

    return interface

In [ ]:
def main():
    # Load all models first
    load_models()
    # Create and launch Gradio interface
    interface = create_interface()
    interface.launch()


if __name__ == "__main__":
    main()